# TTP

In [179]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [180]:
%pip install openpyxl

df = pd.read_excel('../data/processed/DB People.xlsx', sheet_name='DB')

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [181]:
#Visualizar que la base de datos queda bien cargada
df.head()

,GERENCIA,Clave,Nombre,Organización,Departamento,Fecha,Hora,Terminal,Tipo,Tipo Acceso
0,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-03,07:36:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada
1,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-03,17:17:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida
2,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,05:02:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada
3,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,06:02:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida
4,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,09:58:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada


In [182]:
#voy a eliminar las columnas "Organización","Departamento"
df = df.drop(columns=['Organización', 'Departamento'])
df

,GERENCIA,Clave,Nombre,Fecha,Hora,Terminal,Tipo,Tipo Acceso
0,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-03,07:36:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada
1,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-03,17:17:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida
2,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-04,05:02:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada
3,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-04,06:02:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida
4,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-04,09:58:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada
...,...,...,...,...,...,...,...,...
4747,People,38613828,MARIANA SANCHEZ QUIROGA,2025-06-11,17:18:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida
4748,People,38613828,MARIANA SANCHEZ QUIROGA,2025-06-12,07:40:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada
4749,People,38613828,MARIANA SANCHEZ QUIROGA,2025-06-12,14:11:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida
4750,People,38613828,MARIANA SANCHEZ QUIROGA,2025-06-12,14:14:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada


In [183]:
# Combinar Fecha y Hora en un solo datetime, asegurando que Hora sea string
df["FechaHora"] = pd.to_datetime(
	df["Fecha"].dt.strftime('%Y-%m-%d') + " " + df["Hora"].apply(lambda x: x.strftime('%H:%M:%S') if not isinstance(x, str) else x)
)
df

,GERENCIA,Clave,Nombre,Fecha,Hora,Terminal,Tipo,Tipo Acceso,FechaHora
0,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-03,07:36:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,2025-06-03 07:36:00
1,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-03,17:17:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,2025-06-03 17:17:00
2,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-04,05:02:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,2025-06-04 05:02:00
3,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-04,06:02:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,2025-06-04 06:02:00
4,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-04,09:58:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,2025-06-04 09:58:00
...,...,...,...,...,...,...,...,...,...
4747,People,38613828,MARIANA SANCHEZ QUIROGA,2025-06-11,17:18:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,2025-06-11 17:18:00
4748,People,38613828,MARIANA SANCHEZ QUIROGA,2025-06-12,07:40:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,2025-06-12 07:40:00
4749,People,38613828,MARIANA SANCHEZ QUIROGA,2025-06-12,14:11:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,2025-06-12 14:11:00
4750,People,38613828,MARIANA SANCHEZ QUIROGA,2025-06-12,14:14:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,2025-06-12 14:14:00


In [184]:
df = df.drop(columns=['Fecha', 'Hora'])

In [185]:
# Normalizamos
df["Tipo Acceso"] = df["Tipo Acceso"].str.upper()

# Creamos columna con la fecha (sin hora)
df["FechaSolo"] = df["FechaHora"].dt.date

# Obtenemos la primera entrada y última salida por persona y día
entradas_min = (
    df[df["Tipo Acceso"] == "ENTRADA"]
    .sort_values("FechaHora")
    .groupby(["Clave", "FechaSolo"], as_index=False)
    .first()
)

salidas_max = (
    df[df["Tipo Acceso"] == "SALIDA"]
    .sort_values("FechaHora")
    .groupby(["Clave", "FechaSolo"], as_index=False)
    .last()
)

# Unimos ambos resultados
df_filtrado = pd.concat([entradas_min, salidas_max], ignore_index=True)
df_filtrado = df_filtrado.sort_values(by=["Clave", "FechaHora"]).reset_index(drop=True)

print(df_filtrado.head())

      Clave   FechaSolo GERENCIA                       Nombre  \
0  38600035  2025-06-03  Calidad  CLARA LIGEIA RUIZ HERNANDEZ   
1  38600035  2025-06-03  Calidad  CLARA LIGEIA RUIZ HERNANDEZ   
2  38600035  2025-06-04  Calidad  CLARA LIGEIA RUIZ HERNANDEZ   
3  38600035  2025-06-04  Calidad  CLARA LIGEIA RUIZ HERNANDEZ   
4  38600035  2025-06-05  Calidad  CLARA LIGEIA RUIZ HERNANDEZ   

                            Terminal        Tipo Tipo Acceso  \
0  Tibasosa Der Entrada Contratistas  Asistencia     ENTRADA   
1   Tibasosa Izq Salida Funcionarios  Asistencia      SALIDA   
2  Tibasosa Der Entrada Contratistas  Asistencia     ENTRADA   
3   Tibasosa Izq Salida Funcionarios  Asistencia      SALIDA   
4   Tibasosa Izq Salida Funcionarios  Asistencia      SALIDA   

            FechaHora  
0 2025-06-03 07:36:00  
1 2025-06-03 17:17:00  
2 2025-06-04 05:02:00  
3 2025-06-04 06:02:00  
4 2025-06-05 17:33:00  


In [186]:

grupos = []
for (clave, fecha), grupo in df_filtrado.groupby([df_filtrado["Clave"], df_filtrado["FechaHora"].dt.date]):
    grupo = grupo.sort_values("FechaHora")
    nombre = grupo["Nombre"].iloc[0]  # nombre de la persona
    gerencia = grupo["GERENCIA"].iloc[0]  # gerencia de la persona

    entradas = grupo.loc[grupo["Tipo Acceso"] == "ENTRADA", "FechaHora"].tolist()
    salidas  = grupo.loc[grupo["Tipo Acceso"] == "SALIDA", "FechaHora"].tolist()

    # Como ya filtramos, solo habrá máx. una entrada y una salida por día
    e = entradas[0] if len(entradas) > 0 else None
    s = salidas[0]  if len(salidas)  > 0 else None

    imputado = []
    # Si falta entrada → crear 8h antes de la salida
    if e is None and s is not None:
        e = s - pd.Timedelta(hours=8)
        imputado.append("Entrada")

    # Si falta salida → crear 8h después de la entrada
    if s is None and e is not None:
        s = e + pd.Timedelta(hours=8)
        imputado.append("Salida")

    grupos.append({
        "Gerencia": gerencia,
        "Clave": clave,
        "Nombre": nombre,
        "Fecha": fecha,
        "Entrada": e,
        "Salida": s,
        "Imputado": ", ".join(imputado) if imputado else "No"
    })

# Creamos el DataFrame
parejas = pd.DataFrame(grupos)

# Calculamos horas trabajadas
parejas["HorasTrabajadas"] = (parejas["Salida"] - parejas["Entrada"]).dt.total_seconds() / 3600

print(parejas.head())

  Gerencia     Clave                       Nombre       Fecha  \
0  Calidad  38600035  CLARA LIGEIA RUIZ HERNANDEZ  2025-06-03   
1  Calidad  38600035  CLARA LIGEIA RUIZ HERNANDEZ  2025-06-04   
2  Calidad  38600035  CLARA LIGEIA RUIZ HERNANDEZ  2025-06-05   
3  Calidad  38600035  CLARA LIGEIA RUIZ HERNANDEZ  2025-06-06   
4  Calidad  38600035  CLARA LIGEIA RUIZ HERNANDEZ  2025-06-09   

              Entrada              Salida Imputado  HorasTrabajadas  
0 2025-06-03 07:36:00 2025-06-03 17:17:00       No         9.683333  
1 2025-06-04 05:02:00 2025-06-04 06:02:00       No         1.000000  
2 2025-06-05 09:33:00 2025-06-05 17:33:00  Entrada         8.000000  
3 2025-06-06 07:36:00 2025-06-06 18:00:00       No        10.400000  
4 2025-06-09 06:34:00 2025-06-09 16:57:00       No        10.383333  


In [187]:
parejas

,Gerencia,Clave,Nombre,Fecha,Entrada,Salida,Imputado,HorasTrabajadas
0,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-03,2025-06-03 07:36:00,2025-06-03 17:17:00,No,9.683333
1,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-04,2025-06-04 05:02:00,2025-06-04 06:02:00,No,1.000000
2,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-05,2025-06-05 09:33:00,2025-06-05 17:33:00,Entrada,8.000000
3,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-06,2025-06-06 07:36:00,2025-06-06 18:00:00,No,10.400000
4,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-09,2025-06-09 06:34:00,2025-06-09 16:57:00,No,10.383333
...,...,...,...,...,...,...,...,...
2446,People,38613828,MARIANA SANCHEZ QUIROGA,2025-06-06,2025-06-06 07:39:00,2025-06-06 14:45:00,No,7.100000
2447,People,38613828,MARIANA SANCHEZ QUIROGA,2025-06-09,2025-06-09 07:45:00,2025-06-09 17:24:00,No,9.650000
2448,People,38613828,MARIANA SANCHEZ QUIROGA,2025-06-10,2025-06-10 07:44:00,2025-06-10 17:18:00,No,9.566667
2449,People,38613828,MARIANA SANCHEZ QUIROGA,2025-06-11,2025-06-11 10:12:00,2025-06-11 17:18:00,No,7.100000


In [188]:
#Tengo un problema con los cambios de día

# 🔧 Ajuste: si la salida es menor que la entrada, significa que es del día siguiente
mask = parejas["Salida"] < parejas["Entrada"]
parejas.loc[mask, "Salida"] = parejas.loc[mask, "Salida"] + pd.Timedelta(days=1)

# Calculamos horas trabajadas
parejas["HorasTrabajadas"] = (parejas["Salida"] - parejas["Entrada"]).dt.total_seconds() / 3600

print(parejas.head())


  Gerencia     Clave                       Nombre       Fecha  \
0  Calidad  38600035  CLARA LIGEIA RUIZ HERNANDEZ  2025-06-03   
1  Calidad  38600035  CLARA LIGEIA RUIZ HERNANDEZ  2025-06-04   
2  Calidad  38600035  CLARA LIGEIA RUIZ HERNANDEZ  2025-06-05   
3  Calidad  38600035  CLARA LIGEIA RUIZ HERNANDEZ  2025-06-06   
4  Calidad  38600035  CLARA LIGEIA RUIZ HERNANDEZ  2025-06-09   

              Entrada              Salida Imputado  HorasTrabajadas  
0 2025-06-03 07:36:00 2025-06-03 17:17:00       No         9.683333  
1 2025-06-04 05:02:00 2025-06-04 06:02:00       No         1.000000  
2 2025-06-05 09:33:00 2025-06-05 17:33:00  Entrada         8.000000  
3 2025-06-06 07:36:00 2025-06-06 18:00:00       No        10.400000  
4 2025-06-09 06:34:00 2025-06-09 16:57:00       No        10.383333  


In [189]:
parejas

,Gerencia,Clave,Nombre,Fecha,Entrada,Salida,Imputado,HorasTrabajadas
0,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-03,2025-06-03 07:36:00,2025-06-03 17:17:00,No,9.683333
1,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-04,2025-06-04 05:02:00,2025-06-04 06:02:00,No,1.000000
2,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-05,2025-06-05 09:33:00,2025-06-05 17:33:00,Entrada,8.000000
3,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-06,2025-06-06 07:36:00,2025-06-06 18:00:00,No,10.400000
4,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,2025-06-09,2025-06-09 06:34:00,2025-06-09 16:57:00,No,10.383333
...,...,...,...,...,...,...,...,...
2446,People,38613828,MARIANA SANCHEZ QUIROGA,2025-06-06,2025-06-06 07:39:00,2025-06-06 14:45:00,No,7.100000
2447,People,38613828,MARIANA SANCHEZ QUIROGA,2025-06-09,2025-06-09 07:45:00,2025-06-09 17:24:00,No,9.650000
2448,People,38613828,MARIANA SANCHEZ QUIROGA,2025-06-10,2025-06-10 07:44:00,2025-06-10 17:18:00,No,9.566667
2449,People,38613828,MARIANA SANCHEZ QUIROGA,2025-06-11,2025-06-11 10:12:00,2025-06-11 17:18:00,No,7.100000


In [190]:
# Definimos función para calcular horas por turno
def calcular_horas_por_turno(row):
    entrada = row["Entrada"]
    salida = row["Salida"]

    # Inicializamos horas
    horas_turno1 = horas_turno2 = horas_turno3 = 0

    # Vamos avanzando desde la entrada hasta la salida por intervalos de turnos
    actual = entrada
    while actual < salida:
        dia_actual = actual.date()

        # Definimos los límites de los turnos para ese día
        t1_inicio = pd.Timestamp(dia_actual) + pd.Timedelta(hours=0)
        t1_fin    = pd.Timestamp(dia_actual) + pd.Timedelta(hours=8)
        t2_inicio = t1_fin
        t2_fin    = pd.Timestamp(dia_actual) + pd.Timedelta(hours=16)
        t3_inicio = t2_fin
        t3_fin    = pd.Timestamp(dia_actual) + pd.Timedelta(hours=24)

        # Calculamos intersecciones
        if actual < t1_fin:  # Parte que cae en turno 1
            fin = min(salida, t1_fin)
            horas_turno1 += (fin - actual).total_seconds() / 3600
            actual = fin

        elif actual < t2_fin:  # Parte que cae en turno 2
            fin = min(salida, t2_fin)
            horas_turno2 += (fin - actual).total_seconds() / 3600
            actual = fin

        elif actual < t3_fin:  # Parte que cae en turno 3
            fin = min(salida, t3_fin)
            horas_turno3 += (fin - actual).total_seconds() / 3600
            actual = fin

    return pd.Series({
        "HorasTurno_00_08": horas_turno1,
        "HorasTurno_08_16": horas_turno2,
        "HorasTurno_16_24": horas_turno3
    })

# Aplicamos a todo el DataFrame
turnos = parejas.apply(calcular_horas_por_turno, axis=1)
parejas = pd.concat([parejas, turnos], axis=1)

print(parejas[["Clave","Nombre","Fecha","Entrada","Salida",
               "HorasTrabajadas","HorasTurno_00_08","HorasTurno_08_16","HorasTurno_16_24"]].head())


      Clave                       Nombre       Fecha             Entrada  \
0  38600035  CLARA LIGEIA RUIZ HERNANDEZ  2025-06-03 2025-06-03 07:36:00   
1  38600035  CLARA LIGEIA RUIZ HERNANDEZ  2025-06-04 2025-06-04 05:02:00   
2  38600035  CLARA LIGEIA RUIZ HERNANDEZ  2025-06-05 2025-06-05 09:33:00   
3  38600035  CLARA LIGEIA RUIZ HERNANDEZ  2025-06-06 2025-06-06 07:36:00   
4  38600035  CLARA LIGEIA RUIZ HERNANDEZ  2025-06-09 2025-06-09 06:34:00   

               Salida  HorasTrabajadas  HorasTurno_00_08  HorasTurno_08_16  \
0 2025-06-03 17:17:00         9.683333          0.400000              8.00   
1 2025-06-04 06:02:00         1.000000          1.000000              0.00   
2 2025-06-05 17:33:00         8.000000          0.000000              6.45   
3 2025-06-06 18:00:00        10.400000          0.400000              8.00   
4 2025-06-09 16:57:00        10.383333          1.433333              8.00   

   HorasTurno_16_24  
0          1.283333  
1          0.000000  
2       

In [191]:
#Ahora quiero guardar un excel con la información procesada
parejas.to_excel('../data/processed/Horas_Personas.xlsx', index=False)